In [5]:
from create_csv import get_all_data

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

final_data, clicks = get_all_data()

def plot_subject_phase(final_data, all_clicks, subject_id: str, selected_phase: str):

    # Find where the x_axis should start and end

    #Grab the specific data we're talking about, this should maybe be abstracted one level above this function
    subject_data = final_data.loc[final_data['groupID'] == subject_id]
    plot_start = subject_data[selected_phase + '-start-time'].values[0]

    last_click = all_clicks[subject_id][selected_phase].max()
    phase_end = subject_data[selected_phase + '-end-time'].values[0]
    plot_end = max(last_click, phase_end)

    fig, ax = plt.subplots()

    ax.plot(all_clicks[subject_id])

    ax.set(xlim=(plot_start, plot_end), ylim=(0, 1))

    plt.show()


# plot_subject_phase(final_data, clicks, 'C-2857', 'beach')

phases = ['city', 'beach', 'airport']

def get_hit_miss(data, clicks, hit_threshold_ms):
    for subject in data:
        for phase in phases:
            subject_clicks_this_phase = clicks[subject['groupID']][phase].tolist()
            phase_start = subject[phase + '-start-time']
            iter_clicks = iter(subject_clicks_this_phase)
            for alert in range(3):
                pass

get_hit_miss(final_data, clicks, 5000)


Error retrieving city-end-time from: 
Object:644057ba4a65a55c7447191b GroupID:A-8558
Error retrieving beach-end-time from: 
Object:644060395db95bbee031bb07 GroupID:B-5983
Error retrieving city-end-time from: 
Object:64406d6a9d88dcc1a74ed547 GroupID:C-366
Error retrieving beach-end-time from: 
Object:6440722c5b80d645c4596202 GroupID:D-3762
Error retrieving city-end-time from: 
Object:64407bcf0a830617c2d6525b GroupID:A-1676
Error retrieving beach-end-time from: 
Object:6442fbfa0139e7f9b5c8c8c5 GroupID:B-2695
Error retrieving city-end-time from: 
Object:6443025f435e57ed9db41922 GroupID:C-7434
Error retrieving beach-end-time from: 
Object:64430932368b5b5fb57a59d7 GroupID:D-9820
Error retrieving beach-end-time from: 
Object:6443143d7166151e87a9d726 GroupID:A-3801
Error retrieving city-end-time from: 
Object:64431a1058018c7549a04eda GroupID:B-4077
Error retrieving beach-end-time from: 
Object:64431e641ac6346e6c038a36 GroupID:C-2484
Error retrieving city-end-time from: 
Object:6446f0d463c0be0

KeyboardInterrupt: 